In [ ]:
from active_critic.utils.gym_utils import (
    make_vec_env, 
    make_dummy_vec_env, 
    sample_expert_transitions_rollouts, 
    make_pomdp_rollouts, 
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew
)
import gym
from stable_baselines3 import PPO
import torch
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.evaluation import evaluate_policy
import warnings
from typing import Any, Dict, Optional, Type, Union

import numpy as np
import torch as th
from gym import spaces
from torch.nn import functional as F

from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from stable_baselines3.common.policies import ActorCriticCnnPolicy, ActorCriticPolicy, BasePolicy, MultiInputActorCriticPolicy
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import explained_variance,  get_schedule_fn

from torch.utils.data import DataLoader
from imitation.algorithms.adversarial import gail 
from imitation.util.networks import RunningNorm
from imitation.rewards.reward_nets import BasicRewardNet
from stable_baselines3.ppo import MlpPolicy
from active_critic.utils.tboard_graphs import TBoardGraphs
from active_critic.model_src.transformer import PositionalEncoding

import copy

from active_critic.utils.gym_utils import make_policy_dict, ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE, ResetCounterWrapper, TimeLimit, StrictSeqLenWrapper, ImitationLearningWrapper
from active_critic.TQC.tqc import TQC
from active_critic.TQC.tqc_policy import TQCPolicyEval

def run_experiment(device):
    pass
device='cuda'
lookup_freq = 1
env, vec_expert = make_dummy_vec_env(name='pickplace', seq_len=200)
val_env, _ = make_dummy_vec_env(name='pickplace', seq_len=200)

transitions, rollouts = sample_expert_transitions_rollouts(vec_expert.predict, env, 10)
env.envs[0].reset_count = 0
pomdp_rollouts = make_pomdp_rollouts(rollouts, lookup_frq=lookup_freq, count_dim=10)
pomdp_transitions = rollout.flatten_trajectories(pomdp_rollouts)

pomdp_env, pomdp_vec_expert = make_dummy_vec_env_pomdp(name='pickplace', seq_len=200, lookup_freq=lookup_freq)

tqc_learner = TQC(policy='MlpPolicy', env=pomdp_env, device=device)

#opt_kwargs = {'lr':1e-5}
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=pomdp_transitions,
    device=device,
    policy=tqc_learner.policy)

tboard = TBoardGraphs(logname='BC TQC pickplace lookup det'+str(lookup_freq) , data_path='/data/bing/hendrik/gboard/')
def get_avr_succ_rew_det(env, learner, epsiodes):
    success = []
    rews = []
    for i in range(epsiodes):
        obs = env.reset()
        done = False
        while not done:
            action, _ = learner.predict(obs, deterministic=True)
            obs, rew, done, info = env.step(action)
            rews.append(rew)
            if info[0]['success'] > 0:
                success.append(info[0]['success'])
                break
            if done:
                success.append(0)
    return np.array(success), np.array(rews)

for i in range(10000):
    bc_trainer.train(n_epochs=20)
    success, rews = get_avr_succ_rew(env=pomdp_env, learner=bc_trainer.policy, epsiodes=200)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=i)
    tboard.addTrainScalar('Success Rate', value=th.tensor(success.mean()), stepid=i)
    

In [ ]:
tqc_learner.save('tqc_test')

In [ ]:
tqc_learner.load('tqc_test')

In [9]:
success, rews = get_avr_succ_rew_det(env=pomdp_env, learner=tqc_learner.policy, epsiodes=200)

In [17]:
print(success.mean())

0.01


In [14]:
print(success.mean())

0.01


In [16]:
rews

array([[2.2852957e-02],
       [2.3167025e-02],
       [2.3714440e-02],
       ...,
       [1.3438389e-05],
       [3.5262790e-06],
       [6.4357708e-05]], dtype=float32)

In [ ]:
tboard = TBoardGraphs(logname='BC TQC pickplace continue 17.5 lookup '+str(lookup_freq) , data_path='/data/bing/hendrik/gboard/')

In [ ]:
pomdp_env_eval, _ = make_dummy_vec_env_pomdp(name='pickplace', seq_len=200, lookup_freq=lookup_freq)
pomdp_env.envs[0].reset_count = 0
for i in range(10000):
    tqc_learner.learn(total_timesteps=200, log_interval=100000)    
    success, rews = get_avr_succ_rew_det(env=pomdp_env_eval, learner=tqc_learner.policy, epsiodes=200)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=pomdp_env.envs[0].reset_count)
    tboard.addTrainScalar('Success Rate', value=th.tensor(success.mean()), stepid=.reset_count)

In [ ]:
for i in range(10000):
    bc_trainer.train(n_epochs=20)
    success, rews = get_avr_succ_rew(env=pomdp_env, learner=bc_trainer.policy, epsiodes=200)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=i)
    tboard.addTrainScalar('Success Rate', value=th.tensor(success.mean()), stepid=i)